In [1]:
#Imported:
import pandas as pd
import re
import numpy as np

In [2]:
#Functions to use: 
def frequency_table(data_table,variable):
    return pd.crosstab(index=data_table[variable],columns="count") 

def nothing_is_nan (data_table,variable):
    index=0
    for i in data_table[variable]:
        if i=='':
            data_table[variable][index]=np.nan           
        index+=1

def delete_character(data_table,variable,to_delete):
    data_table[variable]=data_table[variable].str.replace(to_delete, '')

def change_for (data_table,variable,to_change, new_value):
    data_table[variable]=data_table[variable].str.replace(to_change,new_value)

In [3]:
#Importing and Inspecting the data

In [4]:
data = pd.read_csv('./GSAF5.csv', encoding = "ISO-8859-1")
data.head(30)


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
5,2016.09.15.R,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,NaN,NaN
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,NaN,NaN
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,NaN,NaN
8,2016.09.06,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,NaN,NaN
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN


In [5]:
name_columns=list(data.columns.values)
print(name_columns)

['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']


In [6]:
data.dtypes

Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
pdf                       object
href formula              object
href                      object
Case Number.1             object
Case Number.2             object
original order             int64
Unnamed: 22               object
Unnamed: 23               object
dtype: object

In [7]:
null_cols = data.isnull().sum()
print(null_cols)

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
pdf                          0
href formula                 1
href                         3
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64


In [8]:
print("the number of registers is " + str(data.shape[0]))

the number of registers is 5992


In [9]:
#Correcting the colums names in data
name_columns_corrected=[]
for i in name_columns:
    name_columns_corrected.append(i.strip())
data.columns=name_columns_corrected
columns_d=data.columns.values
print(columns_d)

['Case Number' 'Date' 'Year' 'Type' 'Country' 'Area' 'Location' 'Activity'
 'Name' 'Sex' 'Age' 'Injury' 'Fatal (Y/N)' 'Time' 'Species'
 'Investigator or Source' 'pdf' 'href formula' 'href' 'Case Number.1'
 'Case Number.2' 'original order' 'Unnamed: 22' 'Unnamed: 23']


In [10]:
#Dropping the wrong columns (more than 2/3 are null)
drop_cols = list(null_cols[null_cols > data.shape[0]*2/3].index)
print(drop_cols)
data=data.drop(drop_cols, axis=1)

['Unnamed: 22', 'Unnamed: 23']


In [11]:
#Dropping the duplicated columns
if data['Case Number'].all() == data['Case Number.1'].all():
    print(True)
if data['Case Number.1'].all() == data['Case Number.2'].all():
    print(True)

True
True


In [12]:
data=data.drop('Case Number.1', axis=1)
data=data.drop('Case Number.2', axis=1)

In [13]:
"""
There are some differences between the registers regarding how complete are the data.
Most of the information is qualitative
Problem statement: Which is the cause for some registers to be more complete than others?
Initial hypothesis: Whether the data is more or less complete can depend on the temporal 
moment or on some circumstances
"""

'\nThere are some differences between the registers regarding how complete are the data.\nMost of the information is qualitative\nProblem statement: Which is the cause for some registers to be more complete than others?\nInitial hypothesis: Whether the data is more or less complete can depend on the temporal \nmoment or on some circumstances\n'

In [14]:
"""Creating a database with the data of interest. Computing resources data (pdf,href) is not taken 
into account (can depend on the temporal moment and bias the analysis)
"""

working_data=data[['Case Number','Date','Year','Type','Country','Area','Location','Activity','Name','Sex','Age','Injury','Fatal (Y/N)','Time','Species','Investigator or Source' ]]
working_data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016"
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016"
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016"
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016"
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016"


In [15]:
"""
Inspecting one by one the variables of my working data, and cleaning in order to be able to
know if the data is more or less complete. (cleaning errors, establishing patterns)
The data cleansing is going to be made in order to know if data is more or less complete, 
rather than to equal the information (if needed, that would be made in the final step only 
with the final considered data)

"""

'\nInspecting one by one the variables of my working data, and cleaning in order to be able to\nknow if the data is more or less complete. (cleaning errors, establishing patterns)\nThe data cleansing is going to be made in order to know if data is more or less complete, \nrather than to equal the information (if needed, that would be made in the final step only \nwith the final considered data)\n\n'

In [16]:
frequency_table(working_data,'Date')

col_0,count
Date,
10-Jan-2009,1
15-Jun-1937,1
16-Jan-1970,1
22-Jul-2013,1
21-Sep-1908,1
02-Jun-1899,1
03-Feb-1914,1
05-Oct-1985,1
10-Jan-1903,1


In [17]:
p_date1="^\S*\s*\t*[0-9]+\-[A-Z]?[a-z]+\-[0-9]+" #00-mm-00 COMPLETE Level 0
p_date2="^\S*\s*\t*-?[A-Z]?[a-z]+\-[0-9]+" # mm-00 COMPLETE Level 1
p_date41="^\S*\s*S?s?ummer\s*[0-9]+" #month 0000 COMPLETE Level 1
p_date42="^\S*\s*W?w?inter\s*[0-9]+" #month 0000  COMPLETE Level 1
p_date43="^\S*\s*F?f?all\s*[0-9]+" #month 0000  COMPLETE Level 1
p_date44="^\S*\s*A?a?utumn\s*[0-9]+" #month 0000  COMPLETE Level 1
p_date6="^\S*\s*\t*B?b?efore\s*\t*[0-9]+\-?[A-Z]?[a-z]+\-?[0-9]+"#Before some day and month COMPLETE Level 1
p_date8="^\S*\s*\t*[A-Z]?[a-z]+-[A-Z]?[a-z]+-[0-9]+"#Between two months Level 2
p_date7="^\S*\s*\t*B?b?efore\s*\t*[A-Z]?[a-z]+\-?[0-9]+"  #Before some month COMPLETE Level 3
p_date3="^\S*\s*\t*[0-9]+" # 0000  only year  COMPLETE Level 3
p_date5="^\S*\s*B?b?efore[a-z]?\s*[0-9]{4,7}" #Before some year  COMPLETE Level 4
p_date9="^\S*\s*\t*^Wo[a-z]?rld War II"
#no-patern or only patterns should be looked up one by one

"""
To see what patterns are lacking:
for i in working_data["Date"][0:1000]:
    if not re.findall(pattern1,i) and not re.findall(pattern2,i).....:
        print(i)
"""

'\nTo see what patterns are lacking:\nfor i in working_data["Date"][0:1000]:\n    if not re.findall(pattern1,i) and not re.findall(pattern2,i).....:\n        print(i)\n'

In [18]:
delete_character(working_data,'Date','Reporte*d')
delete_character(working_data,'Date','C[irc]*a.')
delete_character(working_data,'Date','of')
delete_character(working_data,'Date','Early')
delete_character(working_data,'Date','Late')
delete_character(working_data,'Date','[Some time ]*b?B?etween\s*')
delete_character(working_data,'Date','No date')
delete_character(working_data,'Date','"')
delete_character(working_data,'Date','Mid')
delete_character(working_data,'Date','\s[or]\s')
delete_character(working_data,'Date',',')



/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [19]:
no_string_pattern=[]
for i in working_data["Date"]:
    if type(i)==str:
        if not re.findall(p_date1,i) and not re.findall(p_date2,i) and not re.findall(p_date3,i)and not re.findall(p_date41,i) and not re.findall(p_date42,i) and not re.findall(p_date43,i) and not re.findall(p_date44,i) and not re.findall(p_date5,i) and not re.findall(p_date6,i) and not re.findall(p_date7,i) and not re.findall(p_date8,i) and not re.findall(p_date9,i):
            no_string_pattern.append(i)    
    else:
        no_string_pattern.append(i)
print(no_string_pattern)
#These are treated one by one (and corrected in next cell)

['Last incident  1994 in Hong Kong', ' May 1993', 'May & Nov-1993', 'Jul- to Sep-1959', 'Apr & Nov-1944', ' to have taken place in 1919', 'Nov- or Dec-1873', 'Sep or Oct-1853', 'Anniversary Day 22-Jan-1850 or 1852', ' in 1847', 'Letter dated 10-Jan-1580', ' late 1960s', '', '', ' After August 1926 and before 1936', 'During the war 1943-1945', 'Before the war', 'Said to be 1941-1945 more likely 1945', 'A few years before 1938', '', '', '', '', ' (3 days after preceding incident) & prior to 19-Jul-1913']


In [20]:
change_for (working_data,'Date','Last incident  1994 in Hong Kong', '1994')
change_for (working_data,'Date',' May 1993','May-1993')
change_for (working_data,'Date','Last incident  1994 in Hong Kong','1994')
change_for (working_data,'Date','May & Nov-1993','May-Nov-1993')
change_for (working_data,'Date','Jul- to Sep-1959','Jul-Sep-1959')
change_for (working_data,'Date',' to have taken place in 1919','1919')
change_for (working_data,'Date','Nov- or Dec-1873','Nov-Dec-1873')
change_for (working_data,'Date','Sep or Oct-1853','Sep-Oct-1853')
change_for (working_data,'Date','Anniversary Day 22-Jan-1850 or 1852','Before 1852')
change_for (working_data,'Date',' in 1847','1847')
change_for (working_data,'Date','Letter dated 10-Jan-1580','10-Jan-1580')
change_for (working_data,'Date',' late 1960s','1960')
change_for (working_data,'Date',' After August 1926 and before 1936','Before 1936')
change_for (working_data,'Date','During the war 1943-1945','World War II')
change_for (working_data,'Date','Before the war','Before 1945')
change_for (working_data,'Date','Said to be 1941-1945 more likely 1945','Before 1945')
change_for (working_data,'Date',' (3 days after preceding incident) & prior to 19-Jul-1913','Before 19-Jul-1913')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [21]:
nothing_is_nan(working_data,'Date')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
frequency_table(working_data,"Year")    

col_0,count
Year,
0,124
5,1
77,1
500,1
1543,1
1554,1
1555,1
1580,1
1595,1


In [23]:
index=0
for i in working_data["Year"]:
    if i<1000:
        working_data["Year"][index]=0
    index+=1

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [24]:
#Zero is nan
index=0
for i in working_data["Year"]:
    if i==0:
        working_data["Year"][index]=np.nan
    index+=1
    
nothing_is_nan (working_data,"Year")

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/site-packages/pandas/core/series.py:1015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value


In [25]:
frequency_table(working_data,"Type")

col_0,count
Type,
Boat,200
Boating,110
Invalid,519
Provoked,557
Sea Disaster,220
Unprovoked,4386


In [26]:
delete_character(working_data,'Type','Invalid')
nothing_is_nan(working_data,"Type")

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
frequency_table(working_data,'Country')

col_0,count
Country,
PHILIPPINES,1
TONGA,3
ADMIRALTY ISLANDS,1
ALGERIA,1
AMERICAN SAMOA,3
ANDAMAN / NICOBAR ISLANDAS,1
ANGOLA,1
ANTIGUA,1
ARGENTINA,1


In [28]:
delete_character(working_data,'Country','?')
delete_character(working_data,'Country','\(UAE\)')
delete_character(working_data,'Country','\s$')
delete_character(working_data,'Country','^\s')

change_for (working_data,'Country','ST. MAARTIN', 'ST. MARTIN')

nothing_is_nan(working_data,'Country') 

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [29]:
frequency_table(working_data,'Area')

col_0,count
Area,
Kikori River mouth,1
La Libertad,1
Lau Province,1
Loyalty Islands,1
Manila Bay,1
New Jersey,1
North Carolina,1
Primorje-Gorski Kotar County,9
"Split-Dalmatia Count,",3


In [30]:
delete_character(working_data,'Area','\s$')
delete_character(working_data,'Area','^\s')
nothing_is_nan(working_data,'Area')
#Area does not seem to need to be more profoundly cleaned for my purposes

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [31]:
frequency_table(working_data,'Location')

col_0,count
Location,
,1
A pearl farm in Roebuck Bay,1
"Ambatolaoka, Nosy Be Island",1
Bellingen,1
"Black Head, south of Taree",1
"Boa Viagem Beach, Recife",1
"Boca de la Leña, La Unión",1
Botany Bay,1
Bunker Bay,1


In [32]:
delete_character(working_data,'Location','\s$')
delete_character(working_data,'Location','^\s')
nothing_is_nan(working_data,'Location')
#Location does not seem to need to be more profoundly cleaned for my purposes

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [33]:
frequency_table(working_data,'Activity')

col_0,count
Activity,
,1
,1
a canoe was pursuing a schooner that had forcibily abducted 5 young girls,1
"""Boat accident""",1
"""Climbing up to ship after repairing the stern in water""",1
"""Crossing the river""",1
"""Flying Tiger"" transport plane went down with 5 men onboard",2
"""Riding waves on a board""",1
"""Swimming vigorously""",1


In [34]:
delete_character(working_data,'Activity','\s$')
delete_character(working_data,'Activity','^\s')
nothing_is_nan(working_data,'Activity')
#Activity does not seem to need to be more profoundly cleaned for my purposes

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid

In [35]:
frequency_table(working_data,"Name")

col_0,count
Name,
Maria Korcsmaros \n,1
2 males,2
"28' sport fishing boat, Bird Dog",1
"7.6 m motorized boat, occupants: fisherman & his son",1
Ben Tillman Homes,1
Braxton Rocha,1
"Captain Jonathan Brown, pilot",1
Grace Darling,1
Morson Daniel,1


In [36]:
#names patterns
delete_character(working_data,'Name','"')
delete_character(working_data,'Name','\s$')
delete_character(working_data,'Name','^\s')

pattern_complete='^\s*\t*\S*[A-Z][a-z]+\s*[A-Z][a-z]+'

nothing_is_nan(working_data,'Name')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [37]:
frequency_table(working_data,"Sex")
#Observo que de female solo hay 500 registros, no hay suficiente paridad, descarto este dato para tenerlo en cuenta

col_0,count
Sex,
.,1
F,585
M,4835
M,2
N,1
lli,1


In [38]:
delete_character(working_data,'Sex',' ')
delete_character(working_data,'Sex','N')
delete_character(working_data,'Sex','.')
delete_character(working_data,'Sex','lli')

nothing_is_nan(working_data,"Sex")

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [39]:
frequency_table(working_data,"Age")

col_0,count
Age,
,2
,1
"""middle-age""",1
"""young""",1
(adult),1
1,2
10,51
10 or 12,1
11,35


In [40]:
working_data["Age"].convert_objects(convert_numeric=True).max()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


87.0

In [41]:
delete_character(working_data,'Age','?')
delete_character(working_data,'Age','X')
delete_character(working_data,'Age','F')
delete_character(working_data,'Age','M')
delete_character(working_data,'Age','M?AKE LINE GREEN')
delete_character(working_data,'Age','A.M.')
delete_character(working_data,'Age','A.M.')
delete_character(working_data,'Age','^\s*')
delete_character(working_data,'Age','A..')
delete_character(working_data,'Age','"')

nothing_is_nan(working_data,'Age')

p_age1="^\S*\s*\t*[1-9]+[months]*" #Complete (aprox, en una limpieza más profunda habría que ver estos)
p_age21="^\S*\s*\t*T?t?een" #Los que solo pone Teen. Poco completo 
p_age22="^\S*\s*\t*Elderly" #Los que solo pone Elderly. Poco completo 
p_age23="^\S*\s*\t*young"  #Los que solo pone young. Poco completo 
p_age24="^\S*\s*\t*adult"  #Los que solo pone adult. Poco completo 
p_age25="^\S*\s*\t*midd?le\s*-?age" #Los que solo pone middle age. Poco completo 



"""          
for i in working_data["Age"]:
    if type(i)==str:
        if not re.findall(p_age1,i) and not re.findall(p_age21,i) and not re.findall(p_age22,i) and not re.findall(p_age23,i) and not re.findall(p_age24,i) and not re.findall(p_age25,i):
            print(i)
"""

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


'          \nfor i in working_data["Age"]:\n    if type(i)==str:\n        if not re.findall(p_age1,i) and not re.findall(p_age21,i) and not re.findall(p_age22,i) and not re.findall(p_age23,i) and not re.findall(p_age24,i) and not re.findall(p_age25,i):\n            print(i)\n'

In [42]:
frequency_table(data,"Injury")

col_0,count
Injury,
"""Struck across loins"" but no injury. According to witnesses, incident involved a bottlenose dolphin.",1
2 of the 5 fishermen were so seriously injured they died of their wounds,1
"3 fingers & thigh lacerated, foot crushed",1
"3 unidentified bodies recovered (male, female & young girl) from gut of female 4.5 m [14'9""] white shark caught in fishing net. They may have drowned in tidal wave following earthquake",1
"55 perished, some were taken by sharks",1
6 lacerations to left hand,1
Ankle lacerated,1
Du Val's leg was bitten but he survived,1
FATAL,1


In [43]:
delete_character(working_data,'Injury','\s$')
delete_character(working_data,'Injury','^\s')
nothing_is_nan(working_data,'Injury')
#Injury does not seem to need to be more profoundly cleaned for my purposes

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [44]:
frequency_table(working_data, 'Fatal (Y/N)')

col_0,count
Fatal (Y/N),
N,8
#VALUE!,1
F,1
N,4315
N,1
UNKNOWN,94
Y,1552
n,1


In [45]:
delete_character(working_data,'Fatal (Y/N)','^\s*')
delete_character(working_data,'Fatal (Y/N)','\s*$')
delete_character(working_data,'Fatal (Y/N)','^\s*UNKNOWN\s*$\t*$')
delete_character(working_data,'Fatal (Y/N)','^\s*#VALUE!\s*$\t*$')
working_data['Fatal (Y/N)']=working_data['Fatal (Y/N)'].str.replace('F', 'Y')
working_data['Fatal (Y/N)']=working_data['Fatal (Y/N)'].str.replace('n', 'N')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

In [46]:
nothing_is_nan (working_data,'Fatal (Y/N)')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [47]:
frequency_table(working_data,'Species')

col_0,count
Species,
,6
"""gummy"" shark (Rhizoprionodon or Loxodon) 1.2 m [4']",1
1.5 m [5'] dusky shark,1
"Blacktip shark, C. maculipinnis. 1.9 m to 2.1 m [6.5' to 7']",1
"Bull shark, 1.2m",1
"Bull shark, 2.5 m",1
"Bull shark, 5'",1
"Bull shark, 8'",1
"Galapagos shark, 6'",1


In [48]:
delete_character(working_data,'Species','^\s')
delete_character(working_data,'Species','^\t')
delete_character(working_data,'Species','\s$')
delete_character(working_data,'Species','\t$')
delete_character(working_data,'Species','"')
nothing_is_nan (working_data,'Species')



/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [49]:
frequency_table(working_data,'Time')

col_0,count
Time,
,2
,1
,1
14h00,1
"""After dark""",1
"""After lunch""",1
"""Early evening""",1
"""Evening""",1
"""Just before 11h00""",1


In [50]:
delete_character(working_data,'Time','"')
delete_character(working_data,'Time','\s')
delete_character(working_data,'Time','\t')
delete_character(working_data,'Time','--')
delete_character(working_data,'Time','Just')
delete_character(working_data,'Time','shortly')
nothing_is_nan (working_data,'Time')


p_time10="\S*\s*\t*[0-9]+h[0-9]+" #complete 1
p_time11="\S*\s*\t*[0-9]+:?j?[0-9]+" #complete 1
p_time20="\S*\s*\t*L?l?unch"   #complete 2
p_time21="\S*\s*\t*E?e?vening"   #complete 2
p_time22="\S*\s*\t*?A?a?fterno?on" #complete 2
p_time23="\S*\s*\t*M?m?orning" #complete 2
p_time24="\S*\s*\t*N?n?ight" #complete 2
p_time25="\S*\s*\t*M?m?idday" #complete 2
p_time26="\S*\s*\t*N?n?oon" #complete 2
p_time27="\S*\s*\t*S?s?set" #complete 2
p_time28="\S*\s*\t*S?s?undown" #complete 2
p_time29="\S*\s*\t*D" #complete 2
p_time30="\S*\s*\t*d" #complete 2
p_time31="\S*\s*\t*After" #complete 2
p_time32="\S*\s*\t*B?b?efore" #complete 2
p_time33="\S*\s*\t*A.?M.?" #complete 3
p_time34="\S*\s*\t*P.M." #complete 3

"""
for i in working_data["Time"]:
    if type(i)==str:
        if not re.findall(p_time10,i) and not re.findall(p_time11,i) and not re.findall(p_time20,i) and not re.findall(p_time21,i) and not re.findall(p_time22,i) and not re.findall(p_time23,i) and not re.findall(p_time24,i) and not re.findall(p_time25,i) and not re.findall(p_time26,i) and not re.findall(p_time27,i) and not re.findall(p_time28,i) and not re.findall(p_time29,i) and not re.findall(p_time30,i) and not re.findall(p_time31,i) and not re.findall(p_time32,i) and not re.findall(p_time33,i) and not re.findall(p_time34,i):
            print(i)
"""

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


'\nfor i in working_data["Time"]:\n    if type(i)==str:\n        if not re.findall(p_time10,i) and not re.findall(p_time11,i) and not re.findall(p_time20,i) and not re.findall(p_time21,i) and not re.findall(p_time22,i) and not re.findall(p_time23,i) and not re.findall(p_time24,i) and not re.findall(p_time25,i) and not re.findall(p_time26,i) and not re.findall(p_time27,i) and not re.findall(p_time28,i) and not re.findall(p_time29,i) and not re.findall(p_time30,i) and not re.findall(p_time31,i) and not re.findall(p_time32,i) and not re.findall(p_time33,i) and not re.findall(p_time34,i):\n            print(i)\n'

In [51]:
frequency_table(working_data,'Investigator or Source')

col_0,count
Investigator or Source,
"Natal Colonist, 8/13/1875; M. Levine, GSAF",1
"San Jose Mercury News, 1/6/1995, p.12A",1
"A. Bleakley; A.M. Rapson, p.148",1
"A. M. Rapson, p.148",1
"A. Sharpe, pp.124-125; Sydney Morning Herald, 2/3/2001 ed.",1
"C. Creswell, GSAF",2
"C. Moore, GSAF",1
"D. Davies, p.185; Natal Daily News, 10/25/1960; Natal Witness, 10/26/1960",1
"E. Razzano & R. Joseph; M. Levine, GSAF",1


In [52]:
delete_character(working_data,'Investigator or Source','\s$')
delete_character(working_data,'Investigator or Source','^\s')
nothing_is_nan(working_data,'Investigator or Source')
#Investigator or Source does not seem to need to be more profoundly cleaned for my purposes

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [53]:
# Inspecting again what is left in working data:
print(working_data.isnull().sum())

Case Number                  0
Date                         6
Year                       127
Type                       519
Country                     43
Area                       402
Location                   496
Activity                   528
Name                       200
Sex                        570
Age                       2691
Injury                      27
Fatal (Y/N)                114
Time                      3223
Species                   2942
Investigator or Source      15
dtype: int64


In [54]:
#Changing nulls to 0 in order to work more conveniently :
working_data=working_data.fillna(0)

In [55]:
"""Creating a new column for each variable that is going to tell me how incomplete is the
information. 1 would be very incomplete, and 0 would be totally complete. 
"""

'Creating a new column for each variable that is going to tell me how incomplete is the\ninformation. 1 would be very incomplete, and 0 would be totally complete. \n'

In [56]:
date_complete=[]
for i in working_data['Date']:
    if type(i)==str:
        if re.findall(p_date1,i):
            date_complete.append(0/6.66)
        elif re.findall(p_date2,i):
            date_complete.append(0/6.66)
        elif re.findall(p_date3,i):
            date_complete.append(3/6.66)
        elif re.findall(p_date41,i):
            date_complete.append(1/6.66)
        elif re.findall(p_date42,i):
            date_complete.append(1/6.66)
        elif re.findall(p_date43,i):
            date_complete.append(1/6.66)
        elif re.findall(p_date44,i):
            date_complete.append(1/6.66)
        elif re.findall(p_date5,i):
            date_complete.append(4/6.66)
        elif re.findall(p_date6,i):
            date_complete.append(1/6.66)
        elif re.findall(p_date7,i):
            date_complete.append(3/6.66)
        elif re.findall(p_date8,i):
            date_complete.append(2/6.66)
        elif re.findall(p_date9,i):
            date_complete.append(5/6.66)
        else:
            date_complete.append(1)
    else:
        date_complete.append(1)
        
working_data['Date Completion'] = date_complete

In [57]:
year_or_not=[]

for i in working_data['Year']:
    if i==0:
        year_or_not.append(1)
    else:
        year_or_not.append(0)
working_data['Year Completion'] = year_or_not      

In [58]:
type_or_not=[]

for i in working_data['Type']:
    if i==0:
        type_or_not.append(1)
    else:
        type_or_not.append(0)
working_data['Type Completion'] = type_or_not    

In [59]:
country_or_not=[]
for i in working_data['Country']:
    if i==0:
        country_or_not.append(1)
    else:
        country_or_not.append(0)
working_data['Country Completion'] = country_or_not   

In [60]:
area_or_not=[]
for i in working_data['Area']:
    if i==0:
        area_or_not.append(1)
    else:
        area_or_not.append(0)
    
working_data['Area Completion'] = area_or_not 

In [61]:
location_or_not=[]
for i in working_data['Location']:
    if i==0:
        location_or_not.append(1)
    else:
        location_or_not.append(0)
working_data['Location Completion'] = location_or_not 

In [62]:
len_activity=[]
for i in working_data['Activity']:
    if type(i)==str:
        len_activity.append(len(i))
    else:
        len_activity.append(0)  
        
working_data['Len Activity'] = len_activity         
lab=[1, 0.75, 0.5, 0.25, 0]
#Binning:
activity_complete= pd.cut(working_data['Len Activity'],5, labels=lab)
working_data['Activity Completion'] = activity_complete

In [63]:
name_complete=[]
for i in working_data['Name']:
    if type(i)==str:
        if re.findall(pattern_complete,i):
            name_complete.append(0)
        else:
            name_complete.append(0.5)
    else:
        name_complete.append(1)
working_data['Name Completion'] = name_complete

In [64]:
sex_or_not=[]
for i in working_data['Sex']:
    if i==0:
        sex_or_not.append(1)
    else:
        sex_or_not.append(0)
working_data['Sex Completion'] = sex_or_not

In [65]:
age_or_not=[]

for i in working_data['Age']:
    if type(i)==str:
        if re.findall(p_age1,i):
            age_or_not.append(0)
        elif re.findall(p_age21,i):
            age_or_not.append(0.5)
        elif re.findall(p_age22,i):
            age_or_not.append(0.5)
        elif re.findall(p_age23,i):
            age_or_not.append(0.5)
        elif re.findall(p_age24,i):
            age_or_not.append(0.5)
        elif re.findall(p_age25,i):
            age_or_not.append(0.5)
        else:
            age_or_not.append(1) 
    else:
        if i==0 or i>100:
            age_or_not.append(1)
        else:
            age_or_not.append(0)

working_data['Age Completion'] = age_or_not

In [66]:
len_injury=[]
for i in working_data["Injury"]:
    if type(i)==str:
        len_injury.append(len(i))
    else:
        len_injury.append(0)
working_data['Len Injury'] = len_injury
#Binning:
injury_complete= pd.cut(working_data['Len Injury'],5, labels=lab)
working_data['Injury Completion'] = injury_complete

In [67]:
fatal_yn_or_not=[]
for i in working_data['Fatal (Y/N)']:
    if i==0:
        fatal_yn_or_not.append(1)
    else:
        fatal_yn_or_not.append(0)
working_data['Fatal Completion'] = fatal_yn_or_not

In [68]:
time_complete=[]
for i in working_data['Time']:
    if type(i)==str:
        if re.findall(p_time10,i):
            time_complete.append(0/3.33)
        elif re.findall(p_time11,i):
            time_complete.append(0/3.33)
        elif re.findall(p_time20,i):
            time_complete.append(1/3.33)
        elif re.findall(p_time21,i):
            time_complete.append(1/3.33)
        elif re.findall(p_time22,i):
            time_complete.append(1/3.33)
        elif re.findall(p_time23,i):
            time_complete.append(1/3.33)
        elif re.findall(p_time24,i):
            time_complete.append(1/3.33)
        elif re.findall(p_time25,i):
            time_complete.append(1/3.33)
        elif re.findall(p_time26,i):
            time_complete.append(1/3.33)
        elif re.findall(p_time27,i):
            time_complete.append(1/3.33)
        elif re.findall(p_time28,i):
            time_complete.append(1/3.33)
        elif re.findall(p_time29,i):
            time_complete.append(1/3.33)
        elif re.findall(p_time30,i):
            time_complete.append(1/3.33)
        elif re.findall(p_time31,i):
            time_complete.append(1/3.33)
        elif re.findall(p_time32,i):
            time_complete.append(1/3.33)
        elif re.findall(p_time33,i):
            time_complete.append(2/3.33)
        elif re.findall(p_time34,i):
            time_complete.append(2/3.33)
        elif i==0:
            time_complete.append(1)
        else:
            time_complete.append(1)
    else:
            time_complete.append(1)
working_data['Time Completion'] = time_complete

In [69]:
shark_complete=[]
Shark_Types=["Bignose", "Blacktip reef", "Blind", "Blue", "Bull", "Cat", "Collared", "Creek whaler", "Crested horn", "Crocodile", "Dogfishes", "Dusky whaler", "Dwarf sawfish", "Eastern angel", "Eastern shovelnose ray", "Freshwater sawfish", "Fossil", "Giant shovelnose ray", "Graceful", "Great hammerhead", "Green sawfish", "Grey nurse", "Grey reef", "Grey sharpnose", "Hardnose", "Hound", "Lemon", "Longtail carpet", "Manta ray", "Milk", "Narrow sawfish", "Nervous", "Oceanic whitetip", "Pigeye", "Sandbar", "Scalloped hammerhead", "ray", "Shortfin mako", "Silky", "Silvertip", "Sliteye", "Smooth hammerhead", "Speartooth", "Spinner", "Spot-tail", "Tawny", "Thresher", "Tiger", "Weasel", "Winghead", "Whitecheek", "White", "White spotted guitarfish", "Whitetip reef", "Wobbegong", "Zebra"]

for e in working_data["Species"]:
    for i in Shark_Types:
        if e==0:
            val=1
        else:
            if i.lower() in e.lower(): 
                val=0
            else:
                val=0.5
    shark_complete.append(val)
                
working_data['Type Completion'] = shark_complete

In [70]:
investigator_or_not=[]
for i in working_data['Investigator or Source']:
    if i==0:
        investigator_or_not.append(1)
    else:
        investigator_or_not.append(0)
working_data['Source Completion'] = investigator_or_not

In [71]:
#Adding a column that sums how incomplete is each row (each register): 
Incompletion=['Date Completion', 'Year Completion', 'Type Completion', 'Country Completion', 'Area Completion', 'Location Completion', 'Activity Completion', 'Name Completion', 'Sex Completion', 'Age Completion','Injury Completion', 'Fatal Completion', 'Time Completion', 'Source Completion']
working_data['Incompletion']=working_data[Incompletion].sum(axis=1)


In [72]:
#Checking
name_w_col=list(working_data.columns.values)
print(name_w_col)
working_data.head(30)


['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species', 'Investigator or Source', 'Date Completion', 'Year Completion', 'Type Completion', 'Country Completion', 'Area Completion', 'Location Completion', 'Len Activity', 'Activity Completion', 'Name Completion', 'Sex Completion', 'Age Completion', 'Len Injury', 'Injury Completion', 'Fatal Completion', 'Time Completion', 'Source Completion', 'Incompletion']


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Activity Completion,Name Completion,Sex Completion,Age Completion,Len Injury,Injury Completion,Fatal Completion,Time Completion,Source Completion,Incompletion
0,2016.09.18.c,18-Sep-16,2016.0,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,1.0,0.5,0,0.0,21,1.00,0,0.0000,0,3.5000
1,2016.09.18.b,18-Sep-16,2016.0,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,1.0,0.0,0,0.0,20,1.00,0,0.0000,0,3.0000
2,2016.09.18.a,18-Sep-16,2016.0,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,1.0,0.5,0,0.0,24,1.00,0,0.0000,0,3.5000
3,2016.09.17,17-Sep-16,2016.0,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,1.0,0.0,0,1.0,28,1.00,0,1.0000,0,5.0000
4,2016.09.15,16-Sep-16,2016.0,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,1.0,0.5,0,1.0,37,1.00,0,1.0000,0,5.0000
5,2016.09.15.R,15-Sep-16,2016.0,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,0,...,1.0,0.5,1,1.0,40,1.00,0,1.0000,0,6.5000
6,2016.09.11,11-Sep-16,2016.0,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,1.0,0.5,0,0.0,19,1.00,0,0.0000,0,3.0000
7,2016.09.07,07-Sep-16,2016.0,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,1.0,0.5,0,0.0,40,1.00,0,0.0000,0,3.0000
8,2016.09.06,06-Sep-16,2016.0,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,1.0,0.0,0,0.0,5,1.00,0,0.0000,0,3.0000
9,2016.09.05.b,05-Sep-16,2016.0,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,1.0,0.0,0,0.0,42,1.00,0,0.3003,0,3.3003


In [73]:
#Saving working_data (in case is needed in the future)
working_data.to_csv('./working_data.csv', index=False)

In [74]:
"""Which variables could be used for hypothesis? Not the ones with low frequency (very 
different values that would have to little sampling) or too many nulls.
"""

info="Values are null in more than 40% registers in: "
W_null_cols = working_data.isnull().sum()
columns_null40=list(W_null_cols[W_null_cols > working_data.shape[0]/2.5].index)
print(info + str(columns_null40))

print(data.shape[0])
possible_hypothesis_variables=['Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Injury', 'Fatal (Y/N)', 'Investigator or Source']


info2="Frequency is very low in: "
low_freq=[]
for i in possible_hypothesis_variables:
    if len(frequency_table(working_data,i).to_dict()['count'].keys())>working_data.shape[0]/30:
        low_freq.append(i)
print(info2+str(low_freq))


Values are null in more than 40% registers in: []
5992
Frequency is very low in: ['Year', 'Area', 'Location', 'Activity', 'Injury', 'Investigator or Source']


In [75]:
#Selecting the columns valid for the hypothesis and the completion column. 

final_data=working_data[['Case Number','Year', 'Type', 'Country', 'Fatal (Y/N)','Incompletion']]
print(final_data.dtypes)

Case Number      object
Year            float64
Type             object
Country          object
Fatal (Y/N)      object
Incompletion    float64
dtype: object


In [76]:
#Changing column type in Year
final_data['Year']=final_data['Year'].astype(int)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [77]:
final_data.head()

,Case Number,Year,Type,Country,Fatal (Y/N),Incompletion
0,2016.09.18.c,2016,Unprovoked,USA,N,3.5
1,2016.09.18.b,2016,Unprovoked,USA,N,3.0
2,2016.09.18.a,2016,Unprovoked,USA,N,3.5
3,2016.09.17,2016,Unprovoked,AUSTRALIA,N,5.0
4,2016.09.15,2016,Unprovoked,AUSTRALIA,N,5.0


In [78]:
#Is there more need for cleaning? Some data with very low freq of each case?

info3="Frequency is very low in: "
low_freq2=[]
for i in list(final_data.columns.values):
    if len(frequency_table(final_data,i).to_dict()['count'].keys())>final_data.shape[0]/500:
        low_freq2.append(i)
print(info3+str(low_freq2))



Frequency is very low in: ['Case Number', 'Year', 'Country', 'Incompletion']


In [79]:
"""Low freq is no problem for Year(not categorical but ordinal), Case number(should be unique) 
and Incompletion(is cuantitative). Problem for Country
"""

'Low freq is no problem for Year(not categorical but ordinal), Case number(should be unique) \nand Incompletion(is cuantitative). Problem for Country\n'

In [80]:
#Country final cleaning
index=0
for i in final_data['Country']:
    if i!=0:
        final_data['Country'][index]=i.strip().lower().capitalize()
    index+=1
frequency_table(final_data,'Country')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


col_0,count
Country,
0,43
Admiralty islands,1
Algeria,1
American samoa,3
Andaman / nicobar islandas,1
Angola,1
Antigua,1
Argentina,1
Aruba,1


In [120]:
"""Selecting which ones to use in Country since lots of them have very low freq. I am going
to use only the ones with at least a sample of 20
"""
final_data['Country']

dict_country=frequency_table(final_data,'Country').to_dict()['count']

country_2=[]

for e in dict_country.keys():
    if dict_country[e]<20:
        country_2.append(0)
    else:
        country_2.append(e)

index=0
for i in final_data['Country']: 
    if i in country_2:
        pass 
    else: 
        final_data['Country'][index]="country not for analysis"
    index+=1     

frequency_table(final_data,'Country')

col_0,count
Country,
0,45
Australia,1279
Bahamas,98
Brazil,102
Croatia,34
Cuba,42
Egypt,38
Fiji,65
French polynesia,22


In [121]:
#Dropping duplicates
before = len(final_data)
final_data = final_data.drop_duplicates()
after = len(final_data)
print('Number of duplicate records dropped: ', str(before - after))

Number of duplicate records dropped:  3


In [122]:
#Saving final_data
final_data.to_csv('./final_data.csv', index=False)

In [123]:
final_data.head()

,Case Number,Year,Type,Country,Fatal (Y/N),Incompletion
0,2016.09.18.c,2016,Unprovoked,0,N,3.5
1,2016.09.18.b,2016,Unprovoked,0,N,3.0
2,2016.09.18.a,2016,Unprovoked,Usa,N,3.5
3,2016.09.17,2016,Unprovoked,Australia,N,5.0
4,2016.09.15,2016,Unprovoked,Australia,N,5.0
